<img style='float: left' width="150px" src="http://secoora.org/sites/default/files/secoora_logo.png">
<br><br>
## [SECOORA Notebook 2](http://secoora.org/)

### Sea Surface Height time-series model skill

This notebook calculates several skill scores for the 
SECOORA models weekly time-series saved by [00-fetch_data.ipynb](00-fetch_data.ipynb).

### Load configuration

In [1]:
import os
try:
    import cPickle as pickle
except ImportError:
    import pickle


run_name = '2014-07-07'
fname = os.path.join(run_name, 'config.pkl')
with open(fname, 'rb') as f:
    config = pickle.load(f)

In [2]:
import numpy as np
from pandas import DataFrame, read_csv
from utilities import (load_secoora_ncs, to_html,
                       save_html, apply_skill)


fname = '{}-all_obs.csv'.format(run_name)
all_obs = read_csv(os.path.join(run_name, fname), index_col='name')


def rename_cols(df):
    columns = dict()
    for station in df.columns:
        mask = all_obs['station'] == station
        name = all_obs['station'][mask].index[0]
        columns.update({station: name})
    return df.rename(columns=columns)

### Skill 1: Model Bias (or Mean Bias)

The bias skill compares the model mean elevation against the observations.
Because the observations were saved in NAVD datum, the deviation is usually a
datum difference between the model forcings and the observations.  (This can
be confirmed via the constant bias observed at different runs.)

$$ \text{MB} = \mathbf{\overline{m}} - \mathbf{\overline{o}}$$

In [3]:
from utilities import mean_bias

dfs = load_secoora_ncs(run_name)

df = apply_skill(dfs, mean_bias, remove_mean=False, filter_tides=False)
df = rename_cols(df)
skill_score = dict(mean_bias=df.copy())

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

html = to_html(df.T)
fname = os.path.join(run_name, 'mean_bias.html'.format(run_name))
save_html(fname, html)
html

,COAWST_4,ESTOFS,HYCOM,ROMS_ESPRESSO,USF_FVCOM,USF_ROMS
"Duck, NC",--,0.07,--,--,--,--
"Oregon Inlet Marina, NC",0.19,0.25,0.58,0.46,--,--
"Wrightsville Beach, NC",0.13,0.11,0.44,--,--,--
"Springmaid Pier, SC",0.16,0.14,--,--,--,--
"Oyster Landing (N Inlet Estuary), SC",--,0.25,--,--,--,--
"Charleston, SC",0.10,--,--,--,--,--
"Fernandina Beach, FL",--,0.19,--,--,--,--
"Mayport (Bar Pilots Dock), FL",--,0.14,--,--,--,--
"Trident Pier, FL",--,0.20,--,--,--,--
"Lake Worth Pier, FL",0.32,0.18,0.22,--,--,--


### Skill 2: Central Root Mean Squared Error

Root Mean Squared Error of the deviations from the mean.

$$ \text{CRMS} = \sqrt{\left(\mathbf{m'} - \mathbf{o'}\right)^2}$$

where: $\mathbf{m'} = \mathbf{m} - \mathbf{\overline{m}}$ and $\mathbf{o'} = \mathbf{o} - \mathbf{\overline{o}}$

In [4]:
from utilities import rmse

dfs = load_secoora_ncs(run_name)

df = apply_skill(dfs, rmse, remove_mean=True, filter_tides=False)
df = rename_cols(df)
skill_score['rmse'] = df.copy()

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

html = to_html(df.T)
fname = os.path.join(run_name, 'rmse.html'.format(run_name))
save_html(fname, html)
html

,COAWST_4,ESTOFS,HYCOM,ROMS_ESPRESSO,USF_FVCOM,USF_ROMS
"Duck, NC",--,0.09,--,--,--,--
"Oregon Inlet Marina, NC",0.25,0.34,0.21,0.23,--,--
"Wrightsville Beach, NC",0.16,0.15,0.28,--,--,--
"Springmaid Pier, SC",0.18,0.23,--,--,--,--
"Oyster Landing (N Inlet Estuary), SC",--,0.32,--,--,--,--
"Charleston, SC",0.13,--,--,--,--,--
"Fernandina Beach, FL",--,0.22,--,--,--,--
"Mayport (Bar Pilots Dock), FL",--,0.15,--,--,--,--
"Trident Pier, FL",--,0.07,--,--,--,--
"Lake Worth Pier, FL",0.10,0.09,0.26,--,--,--


### Skill 3: R$^2$

https://en.wikipedia.org/wiki/Coefficient_of_determination

In [5]:
from utilities import r2

dfs = load_secoora_ncs(run_name)

df = apply_skill(dfs, r2, remove_mean=True, filter_tides=False)
df = rename_cols(df)
skill_score['r2'] = df.copy()

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

html = to_html(df.T)
fname = os.path.join(run_name, 'r2.html'.format(run_name))
save_html(fname, html)
html

,COAWST_4,ESTOFS,HYCOM,ROMS_ESPRESSO,USF_FVCOM,USF_ROMS
"Duck, NC",--,0.92,--,--,--,--
"Oregon Inlet Marina, NC",-0.81,-2.35,-3.73,-0.50,--,--
"Wrightsville Beach, NC",0.80,0.82,-0.21,--,--,--
"Springmaid Pier, SC",0.84,0.76,--,--,--,--
"Oyster Landing (N Inlet Estuary), SC",--,0.37,--,--,--,--
"Charleston, SC",0.93,--,--,--,--,--
"Fernandina Beach, FL",--,0.84,--,--,--,--
"Mayport (Bar Pilots Dock), FL",--,0.87,--,--,--,--
"Trident Pier, FL",--,0.94,--,--,--,--
"Lake Worth Pier, FL",0.85,0.89,0.12,--,--,--


### Skill 4: Low passed R$^2$

http://dx.doi.org/10.1175/1520-0450(1979)018%3C1016:LFIOAT%3E2.0.CO;2

https://github.com/ioos/secoora/issues/188

In [6]:
from utilities import r2

dfs = load_secoora_ncs(run_name)

df = apply_skill(dfs, r2, remove_mean=True, filter_tides=True)
df = rename_cols(df)
skill_score['low_pass_r2'] = df.copy()

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

html = to_html(df.T)
fname = os.path.join(run_name, 'low_pass_r2.html'.format(run_name))
save_html(fname, html)
html

,COAWST_4,ESTOFS,HYCOM,ROMS_ESPRESSO,USF_FVCOM,USF_ROMS
"Duck, NC",--,0.86,--,--,--,--
"Oregon Inlet Marina, NC",0.02,-0.56,-23.22,-0.33,--,--
"Wrightsville Beach, NC",0.74,0.13,0.84,--,--,--
"Springmaid Pier, SC",0.78,0.03,--,--,--,--
"Oyster Landing (N Inlet Estuary), SC",--,0.00,--,--,--,--
"Charleston, SC",0.66,--,--,--,--,--
"Fernandina Beach, FL",--,0.26,--,--,--,--
"Mayport (Bar Pilots Dock), FL",--,0.01,--,--,--,--
"Trident Pier, FL",--,0.74,--,--,--,--
"Lake Worth Pier, FL",0.67,0.60,0.45,--,--,--


### Skill 4: Low passed and re-sampled (3H) R$^2$

https://github.com/ioos/secoora/issues/183

In [7]:
from utilities import r2

dfs = load_secoora_ncs(run_name)

# SABGOM dt = 3 hours.
dfs = dfs.swapaxes('items', 'major').resample('3H').swapaxes('items', 'major')

df = apply_skill(dfs, r2, remove_mean=True, filter_tides=False)
df = rename_cols(df)
skill_score['low_pass_resampled_3H_r2'] = df.copy()

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

html = to_html(df.T)
fname = os.path.join(run_name, 'low_pass_resampled_3H_r2.html'.format(run_name))
save_html(fname, html)
html

,COAWST_4,ESTOFS,HYCOM,ROMS_ESPRESSO,USF_FVCOM,USF_ROMS
"Duck, NC",--,0.93,--,--,--,--
"Oregon Inlet Marina, NC",-0.73,-2.21,-3.66,-0.48,--,--
"Wrightsville Beach, NC",0.80,0.84,-0.20,--,--,--
"Springmaid Pier, SC",0.82,0.75,--,--,--,--
"Oyster Landing (N Inlet Estuary), SC",--,0.36,--,--,--,--
"Charleston, SC",0.89,--,--,--,--,--
"Fernandina Beach, FL",--,0.84,--,--,--,--
"Mayport (Bar Pilots Dock), FL",--,0.86,--,--,--,--
"Trident Pier, FL",--,0.96,--,--,--,--
"Lake Worth Pier, FL",0.74,0.87,0.11,--,--,--


### Save scores

In [8]:
fname = os.path.join(run_name, 'skill_score.pkl')
with open(fname,'wb') as f:
    pickle.dump(skill_score, f)

### Normalized Taylor diagrams

The radius is model standard deviation error divided  by observations deviation,
azimuth is arc-cosine of cross correlation (R), and distance to point (1, 0) on the
abscissa is Centered RMS.

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
from utilities.taylor_diagram import TaylorDiagram


def make_taylor(samples):
    fig = plt.figure(figsize=(9, 9))
    dia = TaylorDiagram(samples['std']['OBS_DATA'],
                        fig=fig,
                        label="Observation")
    colors = plt.matplotlib.cm.jet(np.linspace(0, 1, len(samples)))
    # Add samples to Taylor diagram.
    samples.drop('OBS_DATA', inplace=True)
    for model, row in samples.iterrows():
        dia.add_sample(row['std'], row['corr'], marker='s', ls='',
                       label=model)
    # Add RMS contours, and label them.
    contours = dia.add_contours(colors='0.5')
    plt.clabel(contours, inline=1, fontsize=10)
    # Add a figure legend.
    kw = dict(prop=dict(size='small'), loc='upper right')
    leg = fig.legend(dia.samplePoints,
                     [p.get_label() for p in dia.samplePoints],
                     numpoints=1, **kw)
    return fig

In [10]:
dfs = load_secoora_ncs(run_name)

# Bin and interpolate all series to 1 hour.
freq = '1H'
for station, df in list(dfs.iteritems()):
    df = df.resample(freq).interpolate().dropna(axis=1)
    if 'OBS_DATA' in df:
        samples = DataFrame.from_dict(dict(std=df.std(),
                                           corr=df.corr()['OBS_DATA']))
    else:
        continue
    samples[samples < 0] = np.NaN
    samples.dropna(inplace=True)
    if len(samples) <= 2:  # 1 obs 1 model.
        continue
    fig = make_taylor(samples)
    fig.savefig(os.path.join(run_name, '{}.png'.format(station)))
    plt.close(fig)